In [49]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

In [3]:
es_client = Elasticsearch("http://localhost:9200")

In [4]:
es_index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_index_name = "course_questions"

es_client.indices.create(index = es_index_name, body = es_index_setting)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/XsfnxcmtRGCMJ5zcatppQg] already exists')

In [5]:
with open('documents.json', 'rt') as open_doc_param:
    docs_content = json.load(open_doc_param)

In [6]:
flatten_doc = []

for course_content in docs_content:
    for section_set in course_content["documents"]:
        section_set["course"] = course_content["course"]
        flatten_doc.append(section_set)

flatten_doc[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [33]:
for doc in tqdm(flatten_doc):
    es_client.index(index = es_index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [37]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [38]:
search_resp = es_client.search(index = es_index_name, body = search_query)

search_resp

ObjectApiResponse({'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2070, 'relation': 'eq'}, 'max_score': 84.26358, 'hits': [{'_index': 'course_questions', '_id': 'BA6MWJABi0BHoWk645zX', '_score': 84.26358, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course_questions', '_id': 'uA6NWJABi0BHoWk6UJ8B', '_score': 84.26358, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run

In [48]:
context_template = """
Q: {question}
A: {text}
""".strip()

built_context = ""

for qa_set in search_resp['hits']['hits']:
    built_context = built_context + context_template.format(question = qa_set['_source']['question'], text = qa_set['_source']['text']) + "\n\n"

print(built_context)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

built_prompt = prompt_template.format(question = "How do I execute a command in a running docker container?", context = built_context)

print(built_prompt)

len(built_prompt)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
dock

1324

In [51]:
encoder = tiktoken.encoding_for_model("gpt-4o")
token = encoder.encode(built_prompt)

len(token)

298